## Import library

In [ ]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', 500)

## Data Loading

In [ ]:
df = pd.read_csv('df_1999_to_2024.csv')

FileNotFoundError: [Errno 2] No such file or directory: 'df_1999_to_2024.csv'

In [ ]:
df.info()

In [ ]:
df

In [ ]:
df.columns

In [ ]:
# drop the first index column
df.drop(columns=['Unnamed: 0'], inplace=True)
df

In [ ]:
# Building next month status
#df = df.sort_values(by=['Loan Sequence Number', 'Monthly Reporting Period'])
#print(df['Monthly Reporting Period'])

In [ ]:
# to organize the format which will make the conversion easier
df['Monthly Reporting Period'] = df['Monthly Reporting Period'].str[:10]
df['Monthly Reporting Period']
# Convert to datetime while coercing invalid formats
df['Monthly Reporting Period'] = pd.to_datetime(
    df['Monthly Reporting Period'], errors='coerce')

# Format back to date only (as string format YYYY-MM)
df['Monthly Reporting Period'] = df['Monthly Reporting Period'].dt.strftime('%Y-%m')

In [ ]:
df = df.sort_values(['Loan Sequence Number', 'Monthly Reporting Period'])

In [ ]:
print(df['Monthly Reporting Period'].isna().sum())

In [ ]:
df

In [ ]:
# Add a column which represents the loan's next month delinquency status, which will be the
df = df.sort_values(by=['Loan Sequence Number', 'Monthly Reporting Period'], ascending=True)
df['Next Month Status'] = df.groupby('Loan Sequence Number')['Current Loan Delinquency Status'].shift(-1)
print(df['Next Month Status'])

In [ ]:
df['Monthly Reporting Period'][df['Loan Sequence Number'] == 'F99Q10404048']

In [ ]:
df[df['Loan Sequence Number'] == 'F99Q10404048']

In [ ]:
test_1 = df[df['Next Month Status'].isna()]

In [ ]:
test_1

In [ ]:
test_1['Zero Balance Code'].value_counts()

In [ ]:
# create a new column to represent prepayment tag
df['Prepayment tag'] = df['Zero Balance Code'].apply(
    lambda x: 1 if x == 1.0 else 0
)

In [ ]:
df['Prepayment tag'].value_counts()

In [ ]:
# check those record whose Next Month Status is not null but shows prepaid
remove_duplicate = df[(df['Prepayment tag'] == 1) & (df['Next Month Status'].notna())]
remove_duplicate
remove_index = remove_duplicate.index

In [ ]:
# Tried several one and realized that the last one have duplicate
df['Monthly Reporting Period'][df['Loan Sequence Number'] == 'F22Q10579658']

In [ ]:
df = df.drop(remove_index, axis=0)
df.info()

In [ ]:
# save the records which Next Month Status is null value
test_2 = df[df['Next Month Status'].isna()]

In [ ]:
# check the zero balance code for these records
test_2['Zero Balance Code'].value_counts()

In [ ]:
# null value match the prepayment tag
df['Prepayment tag'].value_counts()

In [ ]:
# replace the na with 4 to represent prepayment if zero balance code shows 1.0, else Next Month Status = 3
df['Next Month Status'] = df['Next Month Status'].fillna(
    df.apply(lambda row: 4 if row['Prepayment tag'] == 1.0 else np.nan, axis=1)
)


In [ ]:
df['Next Month Status'].isna().sum()

In [ ]:
# check if they are correctly filled
(df['Prepayment tag'][df['Next Month Status'] == 4] == 1).sum() # match the occurence

In [ ]:
# convert the data type for modeling later
df['Next Month Status'] = df['Next Month Status'].astype('int')

In [ ]:
df.info()

In [ ]:
df['Next Month Status'].value_counts()

In [ ]:
# Because we only have to predict 0, 1, 2, 3 categories
df['Next Month Status'] = df['Next Month Status'].apply(
        lambda x: 3 if x > 4 else x)

In [ ]:
df['Next Month Status'].value_counts()

## Data Cleaning

In [ ]:
# check null value
df.isna().sum()

In [ ]:
# save a copy for reference
df_copy = df.copy()

In [ ]:
# check the columns which have high percentage of null values
# Calculate the percentage of missing values
null_percentage = df.isna().sum() / len(df) * 100
null_percentage

In [ ]:
# First, check those columns which null value percentage higher than 50%
percentage_50 = null_percentage[null_percentage>50]
percentage_50

In [ ]:
# Save the columns in percentage_50 and remove certain columns which might be good to keep
columns_to_drop = percentage_50.index.drop(['Zero Balance Code', 'Borrower Assistance Status Code', 'Delinquency Due to Disaster'])
columns_to_drop

In [ ]:
# drop the columns we decide to drop first
# These columns have more than 50% null values, but we keep 'Zero Balance Code' since it might be a siginificant indicator of delinquency status
df = df.drop(columns=columns_to_drop)

In [ ]:
# Also, check those columns which null value percentage lower 50%
percentage_lower_50 = null_percentage[null_percentage<=50]
percentage_lower_50

In [ ]:
# Number of Borrowers
# Before 2018 Q1, more than 1 borrower would only represent 02
# After 2018 Q1, the number shows the real number of borrowers
# First, check how many unknown borrowers
df[(df['Number of Borrowers'] == 99) | df['Number of Borrowers'].isna()]

In [ ]:
# check loans before 2018Q1 in different Loan Purpose
df_before2018Q1 = df[df['Monthly Reporting Period'] <= '2018-03']
df_before2018Q1.groupby('Loan Purpose')['Number of Borrowers'].value_counts()

In [ ]:
# check loans after 2018Q1 in different Loan Purpose
df_after2018Q1 = df[df['Monthly Reporting Period'] > '2018-03']
df_after2018Q1.groupby('Loan Purpose')['Number of Borrowers'].value_counts()

In [ ]:
# Seems like most of the number of borrowers is 1, so we fill null value and 99 with '1' (also a median)
num_borowers_median = df['Number of Borrowers'].median()
df['Number of Borrowers'] = df['Number of Borrowers'].apply(
        lambda x: num_borowers_median if pd.isna(x) or x == 99 else x)

In [ ]:
df['Number of Borrowers'].value_counts()

In [ ]:
# Credit Score
# If Credit Score > 850 or < 300 would shown 9999, meaning not available
# Fill Credit score with median
credit_score_median = df['Credit Score'].median()
df['Credit Score'] = df['Credit Score'].apply(
        lambda x: credit_score_median if pd.isna(x) or x == 9999 else x)
print(df['Credit Score'].isna().sum())
print((df['Credit Score'] == 9999).sum())

In [ ]:
df['Credit Score'].describe()

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Assuming 'Credit Score' is the column that holds the credit score values in your DataFrame
plt.figure(figsize=(8, 6))

# Create the box plot
sns.boxplot(x=df['Credit Score'])

# Adding labels and title
plt.title('Box Plot of Credit Scores')
plt.xlabel('Credit Score')

# Show the plot
plt.show()

In [ ]:
# Create a new column to represent how good is the credit score group
'''
df['Credit Score Group'] = df['Credit Score'].apply(
    lambda x: 'Poor' if 300 <= x < 580 else
              'Fair' if 580 <= x < 670 else
              'Good' if 670 <= x < 740 else
              'Very Good' if 740 <= x < 800 else
              'Exceptional' if 800 <= x <= 850 else 'Unknown'
)
'''

In [ ]:
#df['Credit Score Group'].value_counts()

In [ ]:
# Original Debt-to-Income (DTI) Ratio
# Create a new column to show the level of DTI (Feature Engineering)
# Using a lambda function to categorize based on DTI ratio
df['Original Debt-to-Income (DTI) Ratio_Level'] = df['Original Debt-to-Income (DTI) Ratio'].apply(
    lambda x: 'Low' if x <= 35 else
              'Middle' if 35 < x <= 49 else
              'High' if 50 <= x <= 49 else
              'Very High' if x == 999 else None
)

#df['Original Debt-to-Income (DTI) Ratio'].describe()
#df['Original Debt-to-Income (DTI) Ratio'] = df['Original Debt-to-Income (DTI) Ratio'].fillna(df['Original Debt-to-Income (DTI) Ratio'].median())
df['Original Debt-to-Income (DTI) Ratio_Level'].value_counts()

In [ ]:
# Fill the null value with "Middle" (Mode)
df['Original Debt-to-Income (DTI) Ratio_Level'] = df['Original Debt-to-Income (DTI) Ratio_Level'].fillna('Middle')
print(df['Original Debt-to-Income (DTI) Ratio_Level'].value_counts())
print(df['Original Debt-to-Income (DTI) Ratio_Level'].isna().sum())

In [ ]:
# tempararily try to remove 'Original Debt-to-Income (DTI) Ratio' column and keep 'Original Debt-to-Income (DTI) Ratio_Level'
df = df.drop('Original Debt-to-Income (DTI) Ratio', axis=1)
df.columns

In [ ]:
# First Time Homebuyer Flag: fill null value with mode
first_time_flag_mode = df['First Time Homebuyer Flag'].mode()[0]
df['First Time Homebuyer Flag'] = df['First Time Homebuyer Flag'].apply(
        lambda x: first_time_flag_mode if pd.isna(x) or x == 'Not Available' else x)
df['First Time Homebuyer Flag'].value_counts()

In [ ]:
# fill "not available" with null value in Loan Purpose
df['Loan Purpose'] = df['Loan Purpose'].fillna('Not Available')
print(df['Loan Purpose'].isna().sum())
df['Loan Purpose'].value_counts()

In [ ]:
# Occupancy Status
print(df['Occupancy Status'].isna().sum())
df['Occupancy Status'].value_counts()

# fill "not available" with null value in Occupancy Status
df['Occupancy Status'] = df['Occupancy Status'].fillna('Not Available')
print(df['Occupancy Status'].isna().sum())
df['Occupancy Status'].value_counts()

In [ ]:
# Channel
print(df['Channel'].isna().sum())
df['Channel'].value_counts()
# fill "not available" with null value in Occupancy Status
df['Channel'] = df['Channel'].fillna('Not Available')
print(df['Channel'].isna().sum())
df['Channel'].value_counts()

In [ ]:
# Property Type
print(df['Property Type'].isna().sum())
df['Property Type'].value_counts()
# fill "not available" with null value in Property Type
df['Property Type'] = df['Property Type'].fillna('Not Available')
print(df['Property Type'].isna().sum())
df['Property Type'].value_counts()

In [ ]:
# Amortization Type (Formerly Product Type)
df['Amortization Type (Formerly Product Type)'].value_counts()
df['Amortization Type (Formerly Product Type)'].isna().sum()
# fill "not available" with null value in Amortization Type (Formerly Product Type)
df['Amortization Type (Formerly Product Type)'] = df['Amortization Type (Formerly Product Type)'].fillna('Not Available')
print(df['Amortization Type (Formerly Product Type)'].isna().sum())
df['Amortization Type (Formerly Product Type)'].value_counts()

In [ ]:
# Amortization Type (Formerly Product Type)
df['Interest Only (I/O) Indicator'].value_counts()
df['Interest Only (I/O) Indicator'].isna().sum()
# fill "not available" with null value in Amortization Type (Formerly Product Type)
df['Interest Only (I/O) Indicator'] = df['Interest Only (I/O) Indicator'].fillna('Not Available')
print(df['Interest Only (I/O) Indicator'].isna().sum())
df['Interest Only (I/O) Indicator'].value_counts()

## From here, have not check each one

In [ ]:
#使用相關欄位推算缺失值：
#Estimated Loan-to-Value (ELTV) 以 Original Combined Loan-to-Value (CLTV) 填補。
df['Estimated Loan-to-Value (ELTV)'] = df['Estimated Loan-to-Value (ELTV)'].fillna(df['Original Combined Loan-to-Value (CLTV)'])

In [ ]:
df[df['Original Combined Loan-to-Value (CLTV)'].isna()]

In [ ]:
df['Original Combined Loan-to-Value (CLTV)'].describe()

In [ ]:
df[df['Estimated Loan-to-Value (ELTV)'] == 999]

In [ ]:
df['Loan Sequence Number']

In [ ]:
df[df['Loan Sequence Number'] == 'F20Q41283775']

In [ ]:
df.isna().sum()

In [ ]:
#Original Combined Loan-to-Value (CLTV)
#這是一個數值型欄位，反映貸款金額相對於房屋價值的比例。這是一個重要的財務指標，用於衡量借款人的風險。
#使用中位數填補，因為中位數對於數據的極端值（如高 CLTV 值）更具穩定性
df['Original Combined Loan-to-Value (CLTV)'] = df['Original Combined Loan-to-Value (CLTV)'].fillna(df['Original Combined Loan-to-Value (CLTV)'].median())
#Mortgage Insurance Percentage (MI %)
#這也是一個數值型欄位，描述貸款中與房屋保險相關的比例。缺失值可能意味著該筆貸款未購買保險。
df['Mortgage Insurance Percentage (MI %)'] = df['Mortgage Insurance Percentage (MI %)'].fillna(0)
#Estimated Loan-to-Value (ELTV)
#這是一個數值型欄位，用於估算貸款金額相對於房屋價值的比例。缺失值可能與 Original Combined Loan-to-Value (CLTV) 相關。
df['Estimated Loan-to-Value (ELTV)'] = df['Estimated Loan-to-Value (ELTV)'].fillna(df['Original Combined Loan-to-Value (CLTV)'].fillna(df['Estimated Loan-to-Value (ELTV)'].median()))
# use median to fill Remaining months to legal maturity (only 2 null values)
df['Remaining Months to Legal Maturity'] = df['Remaining Months to Legal Maturity'].fillna(df['Remaining Months to Legal Maturity'].median())

In [ ]:
# review columns_to_drop
columns_to_drop

In [ ]:
# These columns have been dropped earlier
#df = df.drop(columns=['Step Modification Flag'])
#df = df.drop(columns=['Deferred Payment Plan'])
#df = df.drop(columns=['Net Sales Proceeds'])
#df = df.drop(columns=['Actual Loss Calculation'])
#df = df.drop(columns=['Expenses'])
#df = df.drop(columns=['MI Recoveries'])
#df = df.drop(columns=['Defect Settlement Date'])
#df = df.drop(columns=['Modification Cost'])
#df = df.drop(columns=['Delinquent Accrued Interest'])
#df = df.drop(columns=['HARP Indicator'])

In [ ]:
# Fill missing values for Delinquency Due to Disaster
df['Delinquency Due to Disaster'] = df['Delinquency Due to Disaster'].fillna('N')

# Fill missing values for Borrower Assistance Status Code
df['Borrower Assistance Status Code'] = df['Borrower Assistance Status Code'].fillna('No Assistance')

# Fill missing values for Prepayment Penalty Mortgage (PPM) Flag
df['Prepayment Penalty Mortgage (PPM) Flag'] = df['Prepayment Penalty Mortgage (PPM) Flag'].fillna('N')

# Check missing values after processing
print(df.isna().sum())

In [ ]:
# Handle Zero Balance Code
df['Zero Balance Code'] = df['Zero Balance Code'].fillna('Non zero balance')

In [ ]:
df['Zero Balance Code'] = df['Zero Balance Code'].astype('object')
df['Zero Balance Code'].value_counts()

In [ ]:
print(df.info())

In [ ]:
# Group by the loan identifier and count occurrences of 'Current Status' == 2
occurrence_status_2 = (
    df[df['Current Loan Delinquency Status'] == 2]
    .groupby('Loan Sequence Number')  # Replace with your loan identifier column
    .size()
    .rename('Occurrence_status_2')  # Rename the resulting series
)

# Merge the counts back to the original DataFrame
df = df.merge(occurrence_status_2, on='Loan Sequence Number', how='left')

# Fill NaN values with 0 (loans that never have 'Current Status' == 2)
df['Occurrence_status_2'] = df['Occurrence_status_2'].fillna(0).astype(int)

# Display the updated DataFrame
print(df.head())

In [ ]:
# Step 1: Filter records where Current Loan Delinquency Status is 2
filtered_data = df[df['Current Loan Delinquency Status'] == 2]

# Step 2: Handle missing values in Next Month Status
rows_before = filtered_data.shape[0]
filtered_data = filtered_data.dropna(subset=['Next Month Status'])
rows_after = filtered_data.shape[0]
print(f"Rows before drop: {rows_before}, Rows after drop: {rows_after}, Rows removed: {rows_before - rows_after}")

# Step 3: Check for duplicates and remove if necessary
duplicates = filtered_data.duplicated(subset=['Loan Sequence Number', 'Monthly Reporting Period'])
print(f"Number of duplicates: {duplicates.sum()}")
filtered_data = filtered_data.drop_duplicates(subset=['Loan Sequence Number', 'Monthly Reporting Period'])

# Step 4: Verify no missing values remain
print(filtered_data.isna().sum())

In [ ]:
filtered_data.info()

In [ ]:
#filtered_data = filtered_data.drop(['Loan Sequence Number', 'Monthly Reporting Period'], axis=1)

In [ ]:
# Check if there is any duplicate values
# duplicates = filtered_data.duplicated(subset=['Loan Sequence Number', 'Monthly Reporting Period'])
# print(f"Number of duplicates: {duplicates.sum()}")
# filtered_data = filtered_data.drop_duplicates(subset=['Loan Sequence Number', 'Monthly Reporting Period'])

In [ ]:
# 確認類別型欄位
# Select columns with object data type
categorical_columns = filtered_data.select_dtypes(include=['object']).columns

# Drop the specific columns by filtering out their names
categorical_columns = [col for col in categorical_columns if col not in ['Loan Sequence Number', 'Monthly Reporting Period']]

print(f"Categorical columns: {categorical_columns}")


In [ ]:
# 將類別型欄位轉換為 Dummy Variables
filtered_data_dummies = pd.get_dummies(filtered_data, columns=categorical_columns, drop_first=True)

In [ ]:
filtered_data_dummies.info()

In [ ]:
filtered_data_dummies.columns

In [ ]:
filtered_data_dummies_before_201901 = filtered_data_dummies[filtered_data_dummies['Monthly Reporting Period'] < '2019-01']
filtered_data_dummies_after_201901 = filtered_data_dummies[filtered_data_dummies['Monthly Reporting Period'] >= '2019-01']

In [ ]:
filtered_data_dummies_before_201901_copy = filtered_data_dummies_before_201901.copy()
filtered_data_dummies_after_201901_copy = filtered_data_dummies_after_201901.copy()

In [ ]:
filtered_data_dummies_before_201901 = filtered_data_dummies_before_201901.drop(['Loan Sequence Number', 'Monthly Reporting Period'], axis=1)
filtered_data_dummies_after_201901 = filtered_data_dummies_after_201901.drop(['Loan Sequence Number', 'Monthly Reporting Period'], axis=1)

In [ ]:
filtered_data_dummies_before_201901.info()

In [ ]:
filtered_data_dummies_after_201901.info()

In [ ]:
# Import necessary libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix

In [ ]:
#filtered_data_dummies = filtered_data_dummies[filtered_data_dummies['Current Loan Delinquency Status'] == 2]

In [ ]:
# Assuming 'target' is your target variable and the rest are features
X = filtered_data_dummies_before_201901.drop('Next Month Status', axis=1)  # Feature columns
y = filtered_data_dummies_before_201901['Next Month Status']  # Target variable (loan delinquency categories)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize the feature data (optional but recommended for some models)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Initialize the multinomial logistic regression model
model = LogisticRegression(multi_class='multinomial', solver='lbfgs', max_iter=1000)

# Train the model
model.fit(X_train_scaled, y_train)

# Make predictions
y_pred = model.predict(X_test_scaled)

# Evaluate the model
print("Classification Report:\n", classification_report(y_test, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))

In [ ]:
# Assuming 'target' is your target variable and the rest are features
X = filtered_data_dummies_after_201901.drop('Next Month Status', axis=1)  # Feature columns
y = filtered_data_dummies_after_201901['Next Month Status']  # Target variable (loan delinquency categories)

# Split the data into training and testing sets
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize the feature data (optional but recommended for some models)
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
#X_test_scaled = scaler.transform(X_test)

# Initialize the multinomial logistic regression model
#model = LogisticRegression(multi_class='multinomial', solver='lbfgs', max_iter=1000)

# Train the model
#model.fit(X_train_scaled, y_train)

# Make predictions
y_pred = model.predict(X_scaled)

# Evaluate the model
print("Classification Report:\n", classification_report(y, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y, y_pred))

In [ ]:
# Check the feature importance
# Extract the coefficients for each class
coefficients = model.coef_

# Create a DataFrame to view the feature importance
coeff_df = pd.DataFrame(coefficients, columns=X.columns)

# Display feature importance for each class
coeff_df

In [ ]:
# You can also compute the absolute value of coefficients to see the magnitude of importance
feature_importance = coeff_df.abs().mean(axis=0).sort_values(ascending=False)

# Display the importance of each feature
print("Feature Importance (average absolute coefficient values):")
print(feature_importance)

In [ ]:
feature_importance_top30 = feature_importance.nlargest(30)

In [ ]:
import matplotlib.pyplot as plt

# Plot the absolute values of the coefficients for feature importance
plt.figure(figsize=(10, 6))
feature_importance_top30.sort_values().plot(kind='barh')
plt.title('Feature Importance based on Coefficients (Mean Absolute Value)')
plt.xlabel('Importance')
plt.ylabel('Features')
plt.show()

In [ ]:
remove_feature = feature_importance[feature_importance == 0].index
remove_feature

In [ ]:
filtered_data_dummies_before_201901_2 = filtered_data_dummies_before_201901.drop(remove_feature, axis=1)
filtered_data_dummies_after_201901_2 = filtered_data_dummies_after_201901.drop(remove_feature, axis=1)

In [ ]:
# Assuming 'target' is your target variable and the rest are features
X = filtered_data_dummies_before_201901_2.drop('Next Month Status', axis=1)  # Feature columns
y = filtered_data_dummies_before_201901_2['Next Month Status']  # Target variable (loan delinquency categories)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize the feature data (optional but recommended for some models)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Initialize the multinomial logistic regression model
model = LogisticRegression(multi_class='multinomial', solver='lbfgs', max_iter=1000)

# Train the model
model.fit(X_train_scaled, y_train)

# Make predictions
y_pred = model.predict(X_test_scaled)

# Evaluate the model
print("Classification Report:\n", classification_report(y_test, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))

In [ ]:
# Use all of the training data (before 2019 Jan) and retrain the model
X_train = filtered_data_dummies_before_201901_2.drop('Next Month Status', axis=1)
y_train = filtered_data_dummies_before_201901_2['Next Month Status']

X_test = filtered_data_dummies_after_201901_2.drop('Next Month Status', axis=1)
y_test = filtered_data_dummies_after_201901_2['Next Month Status']

# Standardize the feature data (optional but recommended for some models)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Initialize the multinomial logistic regression model
model = LogisticRegression(multi_class='multinomial', solver='lbfgs', max_iter=1000)

# Train the model
model.fit(X_train_scaled, y_train)

# Make predictions
y_pred = model.predict(X_test_scaled)

# Evaluate the model
print("Classification Report:\n", classification_report(y_test, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))

## Supplemental data

In [ ]:
df_fmhpi = pd.read_csv('fmhpi_master_file.csv')
df_fmhpi.info()

In [ ]:
df_fmhpi['GEO_Type'].unique()
# A Core-Based Statistical Area (CBSA) is a geographic region that includes both metropolitan statistical areas (MSAs)
# and micropolitan statistical areas (MSAs), while an MSA is a type of CBSA

In [ ]:
df_fmhpi_CA = df_fmhpi[df_fmhpi['GEO_Name'] == 'CA']
df_fmhpi_CA['YearMonth'] = df_fmhpi_CA['Year'].astype(str) + '-' + df_fmhpi_CA['Month'].astype(str).str.zfill(2)
df_fmhpi_CA

In [ ]:
df_fmhpi_CA = df_fmhpi_CA.sort_values('YearMonth')

plt.figure(figsize=(12, 6))
sns.lineplot(x='YearMonth', y='Index_SA', data=df_fmhpi_CA, marker='o')

# Customize the plot
plt.title('House Index in California', fontsize=16)
plt.xlabel('Time (Year-Month)', fontsize=12)
plt.ylabel('Index_SA', fontsize=12)
plt.xticks(rotation=45)  # Rotate x-axis labels for better visibility
plt.grid(True)
plt.tight_layout()

# Show the plot
plt.show()

In [ ]:
df_fmhpi_merge_before_201901 = filtered_data_dummies_before_201901_copy.merge(df_fmhpi_CA, \
                         left_on='Monthly Reporting Period', \
                        right_on='YearMonth',\
                        how='left')

In [ ]:
df_fmhpi_merge_before_201901_copy = df_fmhpi_merge_before_201901.copy()

In [ ]:
df_fmhpi_merge_after_201901 = filtered_data_dummies_after_201901_copy.merge(df_fmhpi_CA, \
                         left_on='Monthly Reporting Period', \
                        right_on='YearMonth',\
                        how='left')

In [ ]:
df_fmhpi_merge_after_201901_copy = df_fmhpi_merge_after_201901.copy()

In [ ]:
df_fmhpi_merge_before_201901

In [ ]:
df_fmhpi_merge_after_201901

In [ ]:
df_fmhpi_merge_before_201901 = df_fmhpi_merge_before_201901.drop(['Year', 'Month', 'GEO_Type', 'GEO_Name', 'GEO_Code', 'YearMonth', 'Monthly Reporting Period', 'Loan Sequence Number'], axis=1)
df_fmhpi_merge_before_201901

In [ ]:
df_fmhpi_merge_after_201901 = df_fmhpi_merge_after_201901.drop(['Year', 'Month', 'GEO_Type', 'GEO_Name', 'GEO_Code', 'YearMonth', 'Monthly Reporting Period', 'Loan Sequence Number'], axis=1)
df_fmhpi_merge_after_201901

In [ ]:
df['Loan Sequence Number']

In [ ]:
X = df_fmhpi_merge_before_201901.drop('Next Month Status', axis=1)  # Feature columns
y = df_fmhpi_merge_before_201901['Next Month Status']  # Target variable (loan delinquency categories)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize the feature data (optional but recommended for some models)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Initialize the multinomial logistic regression model
model = LogisticRegression(multi_class='multinomial', solver='lbfgs', max_iter=1000)

# Train the model
model.fit(X_train_scaled, y_train)

# Make predictions
y_pred = model.predict(X_test_scaled)

# Evaluate the model
print("Classification Report:\n", classification_report(y_test, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))

In [ ]:
X_train = df_fmhpi_merge_before_201901.drop('Next Month Status', axis=1)
y_train = df_fmhpi_merge_before_201901['Next Month Status']

X_test = df_fmhpi_merge_after_201901.drop('Next Month Status', axis=1)
y_test = df_fmhpi_merge_after_201901['Next Month Status']

# Standardize the feature data (optional but recommended for some models)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Initialize the multinomial logistic regression model
model = LogisticRegression(multi_class='multinomial', solver='lbfgs', max_iter=1000)

# Train the model
model.fit(X_train_scaled, y_train)

# Make predictions
y_pred = model.predict(X_test_scaled)

# Evaluate the model
print("Classification Report:\n", classification_report(y_test, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))

In [ ]:
# check the feature improtance
coefficients = model.coef_

# Create a DataFrame to view the feature importance
coeff_df = pd.DataFrame(coefficients, columns=X_train.columns)

# Display feature importance for each class
coeff_df

In [ ]:
feature_importance = coeff_df.abs().mean(axis=0).sort_values(ascending=False)

# Display the importance of each feature
print("Feature Importance (average absolute coefficient values):")
print(feature_importance)

In [ ]:
feature_importance_top30 = feature_importance.nlargest(30)

In [ ]:
import matplotlib.pyplot as plt

# Plot the absolute values of the coefficients for feature importance
plt.figure(figsize=(10, 6))
feature_importance_top30.sort_values().plot(kind='barh')
plt.title('Feature Importance based on Coefficients (Mean Absolute Value)')
plt.xlabel('Importance')
plt.ylabel('Features')
plt.show()

## Supplemental data_2: Unemployment Data

In [ ]:
# load the unemployment data (for California)
unemployment = pd.read_csv('CAUR.csv')
unemployment.info()

In [ ]:
unemployment

In [ ]:
unemployment['YearMonth'] = pd.to_datetime(unemployment['DATE']).dt.strftime('%Y-%m')

print(unemployment[['DATE', 'YearMonth']].head())

In [ ]:
# take a look
unemployment = unemployment.sort_values('YearMonth')

plt.figure(figsize=(12, 6))
sns.lineplot(x='YearMonth', y='CAUR', data=unemployment[unemployment['YearMonth'] > '1999-01'], marker='o')

# Customize the plot
plt.title('Unemployment Rate Trend Over Time in California', fontsize=16)
plt.xlabel('Time (Year-Month)', fontsize=12)
plt.ylabel('Unemployment Rate (%)', fontsize=12)
plt.xticks(rotation=45)  # Rotate x-axis labels for better visibility
plt.grid(True)
plt.tight_layout()

# Show the plot
plt.show()

In [ ]:
# Merge the unemployment data into the main training data
df_unemployment_merge_before_201901 = df_fmhpi_merge_before_201901_copy.merge(unemployment, \
                         left_on='Monthly Reporting Period', \
                        right_on='YearMonth',\
                        how='left')

In [ ]:
df_unemployment_merge_before_201901_copy = df_unemployment_merge_before_201901.copy()

In [ ]:
df_unemployment_merge_after_201901 = df_fmhpi_merge_after_201901_copy.merge(unemployment, \
                         left_on='Monthly Reporting Period', \
                        right_on='YearMonth',\
                        how='left')

In [ ]:
df_unemployment_merge_after_201901_copy = df_unemployment_merge_after_201901.copy()

In [ ]:
df_unemployment_merge_before_201901

In [ ]:
df_unemployment_merge_before_201901 = df_unemployment_merge_before_201901.drop(['Year', 'Month', 'GEO_Type', 'GEO_Name', 'GEO_Code', 'DATE', 'YearMonth_x', 'YearMonth_y', 'Monthly Reporting Period', 'Loan Sequence Number'], axis=1)
df_unemployment_merge_before_201901

In [ ]:
df_unemployment_merge_after_201901 = df_unemployment_merge_after_201901.drop(['Year', 'Month', 'GEO_Type', 'GEO_Name', 'GEO_Code', 'DATE', 'YearMonth_x', 'YearMonth_y', 'Monthly Reporting Period', 'Loan Sequence Number'], axis=1)
df_unemployment_merge_after_201901

In [ ]:
X = df_unemployment_merge_before_201901.drop('Next Month Status', axis=1)  # Feature columns
y = df_unemployment_merge_before_201901['Next Month Status']  # Target variable (loan delinquency categories)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize the feature data (optional but recommended for some models)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Initialize the multinomial logistic regression model
model = LogisticRegression(multi_class='multinomial', solver='lbfgs', max_iter=1000)

# Train the model
model.fit(X_train_scaled, y_train)

# Make predictions
y_pred = model.predict(X_test_scaled)

# Evaluate the model
print("Classification Report:\n", classification_report(y_test, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))

In [ ]:
X_train = df_unemployment_merge_before_201901.drop('Next Month Status', axis=1)
y_train = df_unemployment_merge_before_201901['Next Month Status']

X_test = df_unemployment_merge_after_201901.drop('Next Month Status', axis=1)
y_test = df_unemployment_merge_after_201901['Next Month Status']

# Standardize the feature data (optional but recommended for some models)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Initialize the multinomial logistic regression model
model = LogisticRegression(multi_class='multinomial', solver='lbfgs', max_iter=1000)

# Train the model
model.fit(X_train_scaled, y_train)

# Make predictions
y_pred = model.predict(X_test_scaled)

# Evaluate the model
print("Classification Report:\n", classification_report(y_test, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))

## YearMonth

In [ ]:
df_201903 = df_unemployment_merge_after_201901_copy[df_unemployment_merge_after_201901_copy['Monthly Reporting Period'] == '2019-03']

In [ ]:
df_201903.info()

In [ ]:
df_201903.head()

In [ ]:
df_201903 = df_201903.drop(['Year', 'Month', 'GEO_Type', 'GEO_Name', 'GEO_Code', 'DATE', 'YearMonth_x', 'YearMonth_y', 'Monthly Reporting Period', 'Loan Sequence Number'], axis=1)

In [ ]:
X = df_201903.drop('Next Month Status', axis=1)  # Feature columns
y = df_201903['Next Month Status']  # Target variable (loan delinquency categories)


# Standardize the feature data
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Make predictions
y_pred = model.predict(X_scaled)

# Evaluate the model
print("Classification Report:\n", classification_report(y, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y, y_pred))

In [ ]:
from sklearn.metrics import mean_squared_error
import numpy as np

# Calculate RMSE
rmse = np.sqrt(mean_squared_error(y, y_pred))

print(f"Root Mean Squared Error (RMSE): {rmse}")